In [7]:
import pandas as pd
from scipy.optimize import curve_fit
from scipy.integrate import quad
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date

## Reading in the data

In [8]:
#reading the Emission current, pressures
vaclog=pd.read_csv("warmup after power cut", sep="\t")
vaclog.head()

#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
    
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)

#creating a new df by dropping the rows where system was warm
cut_vaclog = vaclog.loc[vaclog["Elapsed time"] >= 3310]
cut_vaclog.head()

C:\Users\etiirine\AppData\Local\Temp\ipykernel_28364\1069277753.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  vaclog=pd.read_csv("warmup after power cut", sep="\t")


,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
479,NaN,06/02/2023 18:53:38,3312.0,65.775,2.080000e-07,9.950000e-07,5.486000e-07,7.416000e-07,1.1,0.0,-105.274,58.473,NaN,NaN
480,NaN,06/02/2023 18:53:45,3319.0,65.775,2.080000e-07,9.960000e-07,5.480000e-07,7.416000e-07,1.1,0.0,-105.387,58.698,NaN,NaN
481,NaN,06/02/2023 18:53:52,3326.0,65.775,2.080000e-07,9.960000e-07,5.486000e-07,7.416000e-07,1.1,0.0,-105.162,58.473,NaN,NaN
482,NaN,06/02/2023 18:53:59,3333.0,65.775,2.080000e-07,9.970000e-07,5.480000e-07,7.416000e-07,1.1,0.0,-105.387,58.698,NaN,NaN
483,NaN,06/02/2023 18:54:05,3339.0,65.775,2.080000e-07,9.970000e-07,5.486000e-07,7.416000e-07,1.1,0.0,-105.162,58.698,NaN,NaN


### Temperature conversion 


In [9]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


### Trendline

In [10]:
#slice the observed data starting from the linear part
print(vaclog["Elapsed time"].shape)
print(vaclog["Barion_2"].shape)
x_slice = vaclog.loc[vaclog["Elapsed time"] >= 40000]
print(x_slice)
y_slice = vaclog["Barion_2"][5766:]
print(y_slice)

#generate new data
x_predict = np.arange(10,1000000,1)
z = np.polyfit(x_slice["Elapsed time"],y_slice,1)
fit = np.poly1d(z)
y_predicted = np.polyval(fit, x_predict)
slope = fit[0]
print(slope)
#plot a y=-x line
x = np.arange(10000,200000,1)
y = -2.5726405582157924e-08*x+1e-8


(206883,)
(206883,)
       Live comments                 Time  Elapsed time  injection 100mbar  \
5767             NaN  07/02/2023 05:05:07       40001.0             65.532   
5768             NaN  07/02/2023 05:05:14       40008.0             65.532   
5769             NaN  07/02/2023 05:05:21       40015.0             65.532   
5770             NaN  07/02/2023 05:05:28       40022.0             65.532   
5771             NaN  07/02/2023 05:05:35       40029.0             65.532   
...              ...                  ...           ...                ...   
206879           NaN  23/02/2023 12:37:07     1449521.0             88.985   
206880           NaN  23/02/2023 12:37:15     1449529.0             88.984   
206881           NaN  23/02/2023 12:37:22     1449536.0             88.985   
206882           NaN  23/02/2023 12:37:31     1449545.0             88.984   
206883           NaN  23/02/2023 12:37:39     1449553.0             88.985   

            Barion_2      Barion_1  DUAL ex

### Plotting

In [11]:
#Plotting BA2, BA1 pressure exc time when warm
plt.figure(figsize=(10,6))
plt.plot(vaclog["Elapsed time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.plot(x_predict,y_predicted,marker="_", markersize=4,label='Fit')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Pumpdown pressure log")
ax = plt.gca()
ax.set_yscale('log')
ax.set_xscale('log')

plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')
plt.ylim(1e-10,1e-6)

plt.show()

In [12]:
# Plotting the temp evolution exc warm part
plt.figure()
plt.plot((vaclog["Elapsed time"]/3600),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')

plt.xlabel('Elapsed time (h)')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("Pumpdown temp log")
plt.yscale('linear')